<a href="https://colab.research.google.com/github/sefeoglu/AE_Parseval_Network/blob/master/src/notebooks/ResNet_Tensorflow_keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Wide ResNet 16_2

In [0]:
import tensorflow.compat.v2 as tf

In [0]:

from tensorflow.compat.v2.keras.models import Model
from tensorflow.compat.v2.keras.layers import  Input, Add, Activation, Dropout, Flatten, Dense
from tensorflow.compat.v2.keras.layers import Convolution2D, MaxPooling2D, AveragePooling2D
from tensorflow.compat.v2.keras.layers import BatchNormalization
from tensorflow.compat.v2.keras.regularizers import  l2
from tensorflow.compat.v2.keras import backend as K
import warnings

warnings.filterwarnings("ignore")
weight_decay = 0.0005

In [0]:
def initial_conv(input):
  x = Convolution2D(16,(3,3),padding="same", kernel_initializer='he_normal',
                    kernel_regularizer=l2(weight_decay),use_bias=False)(input)
  channel_axis = 1 if K.image_data_format() == "channels_first" else -1
  x = BatchNormalization(axis=channel_axis, momentum=0.1, epsilon=1e-5, gamma_initializer='uniform')(x)
  x = Activation('relu')(x)
  return x

In [0]:
def expand_conv(init, base, k, strides = (1,1)):
  x = Convolution2D(base * k, kernel_size=(3,3),padding= "same", strides=strides, kernel_initializer="he_normal", kernel_regularizer=l2(weight_decay),
                    use_bias=False)(init)
  channel_axis = 1 if K.image_data_format() == "channels_first" else -1

  x = BatchNormalization(axis=channel_axis, momentum=0.1, epsilon= 1e-5, gamma_initializer= 'uniform')(x)
  x = Activation('relu')(x)

  x = Convolution2D(base * k, kernel_size=(3,3), padding = 'same', kernel_initializer = 'he_normal',kernel_regularizer=l2(weight_decay),
                    use_bias = False)(x)
  skip = Convolution2D(base * k, kernel_size=(1,1), padding='same', strides=strides, kernel_initializer='he_normal',
                       kernel_regularizer=l2(weight_decay),
                       use_bias = False)(init)
  m = Add()([x, skip])
  return m

In [0]:
def conv1_block(input, k=1, dropout = 0.0):
  init = input
  
  channel_axis = 1 if K.image_data_format() == "channels_first" else -1

  x = BatchNormalization(axis=channel_axis, momentum=0.1, epsilon=1e-5, gamma_initializer='uniform')(input)
  x = Activation('relu')(x)
  x = Convolution2D(16 * k, kernel_size=(3,3), padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(weight_decay),
                    use_bias=False)(x)
  if dropout > 0.0: x = Dropout(dropout)(x)

  x = BatchNormalization(axis=channel_axis, momentum=0.1, epsilon=1e-5, gamma_initializer='uniform')(x)
  x = Activation('relu')(x)
  x = Convolution2D(16 * k, kernel_size=(3,3), padding='same', kernel_initializer='he_normal',kernel_regularizer=l2(weight_decay),
                    use_bias = False)(x)
  m = Add()([init, x])
  return m

In [0]:
def  conv2_block(input, k=1, dropout = 0.0):
  init = input

  channel_axis = 1 if K.image_data_format() == "channels_first" else -1

  x = BatchNormalization(axis=channel_axis, momentum=0.1, epsilon=1e-5, gamma_initializer='uniform')(input)
  x = Activation('relu')(x)
  x = Convolution2D(32 * k, kernel_size=(3,3), padding='same', kernel_initializer='he_normal',
                    kernel_regularizer = l2(weight_decay), use_bias = False)(x)
  
  if dropout > 0.0: x = Dropout(dropout)(x)

  x = BatchNormalization(axis=channel_axis, momentum=0.1, epsilon=1e-5, gamma_initializer='uniform')(x)
  x = Activation('relu')(x)
  x = Convolution2D(32*k, kernel_size=(3,3), padding='same', kernel_initializer='he_normal',
                    kernel_regularizer= l2(weight_decay),
                    use_bias = False)(x)
  
  m = Add()([init, x])
  return m


In [0]:
def conv3_block(input, k=1, dropout=0.0):
  init = input

  channel_axis = 1 if K.image_data_format() == "channels_first" else -1

  x = BatchNormalization(axis=channel_axis, momentum= 0.1, epsilon = 1e-5, gamma_initializer='uniform' )(input)
  x = Activation('relu')(x)
  x = Convolution2D(64 * k, kernel_size=(3,3), padding='same', kernel_initializer='he_normal',
                    kernel_regularizer= l2(weight_decay),
                    use_bias = False)(x)
  
  if dropout > 0.0: x = Dropout(dropout)(x)

  x = BatchNormalization(axis=channel_axis, momentum=0.1, epsilon=1e-5, gamma_initializer='uniform')(x)
  x = Activation('relu')(x)
  x = Convolution2D(64 * k, kernel_size=(3,3), padding='same', kernel_initializer='he_normal',
                    kernel_regularizer=l2(weight_decay),
                    use_bias=False)(x)
  m = Add()([init, x])
  return m


In [0]:
def create_wide_residual_network(input_dim, nb_classes=4, N=2, k=2, dropout = 0.0, verbose=1):
  channel_axis = 1 if K.image_data_format() == "channels_first" else -1

  ip = Input(shape=input_dim)

  x = initial_conv(ip)
  nb_conv = 4
  x = expand_conv(x, 16, k)
  nb_conv +=2

  for i in range(N-1):
    x = conv1_block(x, k, dropout)
    nb_conv +=2
  x = BatchNormalization(axis=channel_axis, momentum=0.1, epsilon=1e-5, gamma_initializer='uniform')(x)
  x = Activation('relu')(x)

  x = expand_conv(x, 32, k, strides=(1, 1))
  nb_conv += 2
  for i in range(N - 1):
    x = conv2_block(x, k, dropout)
    nb_conv += 2
  
  x = BatchNormalization(axis=channel_axis, momentum=0.1, epsilon=1e-5, gamma_initializer='uniform')(x)
  x = Activation('relu')(x)
  
  x = expand_conv(x, 64, k, strides=(1, 1))
  nb_conv += 2
  
  for i in range(N - 1):
    x = conv3_block(x, k, dropout)
    nb_conv += 2
    
  x = BatchNormalization(axis=channel_axis, momentum=0.1, epsilon=1e-5, gamma_initializer='uniform')(x)
  x = Activation('relu')(x)
  
  x = AveragePooling2D((8, 8))(x)
  x = Flatten()(x)
  
  x = Dense(nb_classes, kernel_regularizer=l2(weight_decay), activation='softmax')(x)
  
  model = Model(ip, x)
  
  if verbose: print("Wide Residual Network-%d-%d created." % (nb_conv, k))
  
  return model





In [51]:
if __name__ == "__main__":
    from tensorflow.compat.v2.keras.layers import Input
    from tensorflow.compat.v2.keras.models import Model

    init = (32, 32,1)
    wrn16_2 = create_wide_residual_network(init, nb_classes=4, N=2, k=2, dropout = 0.0)
    wrn16_2.summary()

Wide Residual Network-16-2 created.
Model: "model_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            [(None, 32, 32, 1)]  0                                            
__________________________________________________________________________________________________
conv2d_80 (Conv2D)              (None, 32, 32, 16)   144         input_6[0][0]                    
__________________________________________________________________________________________________
batch_normalization_65 (BatchNo (None, 32, 32, 16)   64          conv2d_80[0][0]                  
__________________________________________________________________________________________________
activation_65 (Activation)      (None, 32, 32, 16)   0           batch_normalization_65[0][0]     
________________________________________________________

In [0]:
import skimage.util, skimage.transform
import numpy as np


def crop(image: np.ndarray, maxshape):
    d = [max(0, image.shape[i] - maxshape[i]) for i in [0, 1]]
    if (np.sum(d) == 0):
        return image
    return image[(d[0] + 1) // 2:image.shape[0] - (d[0] // 2), (d[1] + 1) // 2:
                 image.shape[1] - (d[1] // 2)]


def pad(image: np.ndarray, padding: int):
    pad_width = [[padding] * 2] * 2
    if len(image.shape) == 3: pad_width.append((0, 0))
    return np.pad(image, pad_width, mode='constant')


def pad_to_shape(image: np.ndarray, shape):
    d = [shape[i] - image.shape[i] for i in [0, 1]]
    pad_width = [((d[0] + 1) // 2, d[0] // 2), ((d[1] + 1) // 2, d[1] // 2)]
    if len(image.shape) == 3: pad_width.append((0, 0))
    return np.pad(image, pad_width, mode='constant')


def adjust_shape(image: np.ndarray, shape):
    return pad_to_shape(crop(image, shape), shape)


def resize(image: np.ndarray, shape):
    return skimage.transform.resize(image, shape, anti_aliasing=True)


In [0]:
import numpy as np

def random_crop(im, shape):
    d = [im.shape[i] - shape[i] + 1 for i in [0, 1]]
    d = list(map(np.random.randint, d))
    return im[d[0]:d[0] + shape[0], d[1]:d[1] + shape[1]]


def augment_cifar(im):
    im = pad(im, 4)
    im = random_crop(im, [32,32])
    if np.random.rand() > .5:
        im = np.fliplr(im)
    return im


In [0]:
from sklearn.model_selection import train_test_split
import gzip
import pickle

import numpy as np

In [0]:
def read_data():
    with open("data.pz", 'rb') as file_:
        with gzip.GzipFile(fileobj=file_) as gzf:
            data = pickle.load(gzf, encoding='latin1', fix_imports=True)
    return data
data = read_data()


In [54]:
new_data_X = []
Y_data = []
for row in data:
    new_data_X.append(row['crop'])
    Y_data.append(row['label'])
new_data_X = np.array(new_data_X)
new_data_X.shape

(5722, 68, 100)

In [57]:
X = new_data_X.astype('float32')
X[0].shape

(68, 100)

In [58]:
X = list(map(augment_cifar, X))
X = np.array(X)
X.shape

(5722, 32, 32)

In [0]:
img_rows, img_cols = X[0].shape

# transform data set
if K.image_data_format() == 'channels_first':
    X = X.reshape(X.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    X = X.reshape(X.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

In [24]:
X.shape


(5722, 32, 32, 1)

In [0]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder

labelencoder = LabelEncoder()
y_df = pd.DataFrame(Y_data, columns=['Label'])
y_df['Encoded'] = labelencoder.fit_transform(y_df['Label'])

In [60]:
y_df['Label'].value_counts()

open             1500
closed           1500
partiallyOpen    1376
notVisible       1346
Name: Label, dtype: int64

In [61]:
y_df['Encoded'].value_counts()


2    1500
0    1500
3    1376
1    1346
Name: Encoded, dtype: int64

In [0]:
from tensorflow.compat.v2.keras.utils import to_categorical

y_cat = to_categorical(y_df['Encoded'])

In [0]:
from tensorflow.compat.v2.keras.optimizers import SGD

EPOCHS = 200
BS = 128

In [0]:
sgd = SGD(lr=0.1, momentum=0.9)

In [65]:
wrn16_2.compile(loss="categorical_crossentropy", optimizer=sgd, metrics=["acc"])
print("Finished compiling")


Finished compiling


In [0]:
from tensorflow.compat.v2.keras.utils import to_categorical
from tensorflow.compat.v2.keras.callbacks import Callback, LearningRateScheduler, EarlyStopping


In [0]:
def lr_sch(epoch):
    if epoch < 60:
        return 0.1
    elif epoch < 120:
        return 0.02
    elif epoch < 160:
        return 0.004
    else:
        return 0.0008

# Learning rate scheduler callback
lr_scheduler = LearningRateScheduler(lr_sch)

In [0]:
import tensorflow
generator = tensorflow.compat.v2.keras.preprocessing.image.ImageDataGenerator(rotation_range=10,
                               width_shift_range=5./32,
                               height_shift_range=5./32,)

In [0]:
from sklearn.model_selection import KFold
cv = KFold(n_splits=2, random_state=42, shuffle=False)
hist_list = []
train_set= []
test_set = []
for train_index, test_index in cv.split(X):
    print("Train Index: ", train_index, "\n")
    print("Test Index: ", test_index)
    train_set.append(train_index)
    test_set.append(test_index)
    X_train, X_test, y_train, y_test = X[train_index], X[test_index], y_cat[train_index], y_cat[test_index]
    hist = wrn16_2.fit_generator(generator.flow(X_train, y_train, batch_size=BS), steps_per_epoch=len(X_train) // BS, epochs=EPOCHS,
                   callbacks=[lr_scheduler],
                   validation_data=(X_test, y_test),
                   validation_steps=X_test.shape[0] // BS,)
    hist_list.append(hist)

Train Index:  [2861 2862 2863 ... 5719 5720 5721] 

Test Index:  [   0    1    2 ... 2858 2859 2860]
Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/200
22/22 [==============================] - 9s 401ms/step - loss: 2.4788 - acc: 0.3132 - val_loss: 2.7075 - val_acc: 0.2478 - lr: 0.1000
Epoch 2/200
22/22 [==============================] - 7s 302ms/step - loss: 2.3858 - acc: 0.3465 - val_loss: 2.3950 - val_acc: 0.3387 - lr: 0.1000
Epoch 3/200
22/22 [==============================] - 7s 303ms/step - loss: 2.3268 - acc: 0.3417 - val_loss: 2.3503 - val_acc: 0.3443 - lr: 0.1000
Epoch 4/200
22/22 [==============================] - 7s 303ms/step - loss: 2.2811 - acc: 0.3597 - val_loss: 2.2910 - val_acc: 0.3502 - lr: 0.1000
Epoch 5/200
22/22 [==============================] - 7s 304ms/step - loss: 2.2260 - acc: 0.3659 - val_loss: 2.2878 - val_acc: 0.3279 - lr: 0.1000
Epoch 6/200
22/22 [==============================] - 7s 302ms/step - loss: 2.2028 - acc: 0.36

In [42]:

hist = wrn16_2.fit(X, y_cat, 
                        batch_size=BS, epochs=EPOCHS, verbose=2,
                        validation_split = 0.25,
                         shuffle=False, callbacks=[lr_scheduler])

Epoch 1/200
34/34 - 11s - loss: 2.4929 - acc: 0.2785 - val_loss: 2.3528 - val_acc: 0.3564 - lr: 0.1000
Epoch 2/200
34/34 - 9s - loss: 2.4116 - acc: 0.3151 - val_loss: 2.2979 - val_acc: 0.3711 - lr: 0.1000
Epoch 3/200
34/34 - 9s - loss: 2.3323 - acc: 0.3258 - val_loss: 2.2254 - val_acc: 0.4046 - lr: 0.1000
Epoch 4/200
34/34 - 9s - loss: 2.2581 - acc: 0.3328 - val_loss: 2.1402 - val_acc: 0.4018 - lr: 0.1000
Epoch 5/200
34/34 - 9s - loss: 2.1919 - acc: 0.3421 - val_loss: 2.1109 - val_acc: 0.3620 - lr: 0.1000
Epoch 6/200
34/34 - 9s - loss: 2.1393 - acc: 0.3454 - val_loss: 2.0700 - val_acc: 0.3690 - lr: 0.1000
Epoch 7/200
34/34 - 9s - loss: 2.0807 - acc: 0.3444 - val_loss: 2.0119 - val_acc: 0.3788 - lr: 0.1000
Epoch 8/200
34/34 - 9s - loss: 2.0259 - acc: 0.3482 - val_loss: 2.0175 - val_acc: 0.3620 - lr: 0.1000
Epoch 9/200
34/34 - 9s - loss: 1.9720 - acc: 0.3626 - val_loss: 1.9526 - val_acc: 0.3906 - lr: 0.1000
Epoch 10/200
34/34 - 9s - loss: 1.9238 - acc: 0.3626 - val_loss: 1.9000 - val_acc

KeyboardInterrupt: ignored

In [0]:
""" New model"""

EPOCHS_120 = 120
wrn16_4 = create_wide_residual_network(init, nb_classes=4, N=2, k=2, dropout = 0.0, verbose=1)
  
hist2 = wrn16_4.fit(X_train, y_train_cat, 
                        batch_size=BS, epochs=EPOCHS, verbose=1,
                        validation_data=(X_test,y_test_cat), shuffle=True, callbacks=[lr_scheduler])

In [0]:
#hist = wrn28_10.fit(X_train, to_categorical(y_train_df['New']), validation_split=0.33, epochs=EPOCHS, batch_size=BS, verbose=2)

In [0]:
# construct the training image generator for data augmentation
aug = ImageDataGenerator(rotation_range=20, zoom_range=0.15,
width_shift_range=0.2, height_shift_range=0.2, shear_range=0.15,
horizontal_flip=True, fill_mode="nearest")

In [0]:
hist_1 = wrn28_10.fit_generator(aug.flow(X_train, y_train, batch_size=BS),validation_data=(X_test, y_test),callbacks =[lr_scheduler],verbose=1, epochs=EPOCHS)

In [0]:
wrn28_10.save("model_last.h5")

In [0]:
img = X_train[8]
img.shape

In [0]:
y_train[8]

In [0]:
np.argmax(wrn28_10(image))

In [0]:
y_train_df['Label'])

In [0]:
def preprocess(image):
  image = tf.cast(image, tf.float32)
  image = tf.image.resize(image, (68, 100))
  image = tf.keras.applications.mobilenet_v2.preprocess_input(image)
  image = image[None, ...]
  return image

# Helper function to extract labels from probability vector

In [0]:
import tensorflow as tf
image = tf.convert_to_tensor(img)
image = preprocess(image)
image.shape

In [0]:
probs = wrn28_10.predict(image,batch_size=None,steps=1)

label = tf.one_hot(2, probs.shape[-1])
label = tf.reshape(label, (1, probs.shape[-1]))
label

In [0]:
probs

In [0]:
wrn28_10(image)

In [0]:
np.argmax(probs)

In [0]:
import matplotlib.pyplot as plt

history = hist_1
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()